In [4]:
import time
import pycurl
import urllib
import json
import requests

In [39]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self, callback):
        self.callback = callback
        self.conn = None
        self.buffer = ''
        self.setup_connection()
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self):
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        #data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data={data})
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
       # data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data={data})
        return str(r.status_code) + ' ' + str(r.reason)
        

def extract_price(data):
    return data.decode("utf-8") .split(' ')[1][:-1]

def handle_new_price(data):
    print(extract_price(data))
    
def do_bot_things(data,time_to_sleep):
    while(True):
        
        # 1 get infos
        
        # 2 create inputs
        
        # 3 go through graph
        
        #https://stackoverflow.com/questions/37891954/keras-how-do-i-predict-after-i-trained-a-model
        #prediction = model.predict(np.array(tk.texts_to_sequences(text)))
        #print(prediction)
        
        # 4 depending on output : buy, sell or do nothing
        
        time.sleep(time_to_sleep)

In [40]:
stream = SwissQuoteStream(handle_new_price)
stream.buy(0.00000001)

'400 Bad Request'

In [ ]:
stream = SwissQuoteStream(handle_new_price)
#stream.start()